In [30]:
from flask import Flask, render_template, redirect, url_for, request, session, flash
from werkzeug.security import generate_password_hash, check_password_hash
import import_ipynb
from forms import SignUpForm, LoginForm, ExpenseForm
from db import init_db, query_db, get_db, close_connection

def create_app():
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'your_secret_key'

    @app.teardown_appcontext
    def close_db(error):
        close_connection()

    @app.route('/')
    def index():
        return render_template('index.html')

    @app.route('/signup', methods=['GET', 'POST'])
    def signup():
        form = SignUpForm()
        if form.validate_on_submit():
            hashed_password = generate_password_hash(form.password.data, method='sha256')
            db = get_db()
            db.execute('INSERT INTO users (username, email, password) VALUES (?, ?, ?)', 
                       (form.username.data, form.email.data, hashed_password))
            db.commit()
            flash('Account created successfully!', 'success')
            return redirect(url_for('login'))
        return render_template('signup.html', form=form)

    @app.route('/login', methods=['GET', 'POST'])
    def login():
        form = LoginForm()
        if form.validate_on_submit():
            user = query_db('SELECT * FROM users WHERE email = ?', [form.email.data], one=True)
            if user and check_password_hash(user['password'], form.password.data):
                session['user_id'] = user['id']
                flash('Login successful!', 'success')
                return redirect(url_for('dashboard'))
            else:
                flash('Invalid credentials. Please try again.', 'danger')
        return render_template('login.html', form=form)

    @app.route('/dashboard')
    def dashboard():
        if 'user_id' not in session:
            return redirect(url_for('login'))
        user_id = session['user_id']
        expenses = query_db('SELECT * FROM expenses WHERE user_id = ? ORDER BY created_at DESC', [user_id])
        return render_template('dashboard.html', expenses=expenses)

    @app.route('/add', methods=['GET', 'POST'])
    def add_expense():
        if 'user_id' not in session:
            return redirect(url_for('login'))
        form = ExpenseForm()
        if form.validate_on_submit():
            db = get_db()
            db.execute('INSERT INTO expenses (category, amount, comments, user_id) VALUES (?, ?, ?, ?)',
                       (form.category.data, form.amount.data, form.comments.data, session['user_id']))
            db.commit()
            flash('Expense added successfully!', 'success')
            return redirect(url_for('dashboard'))
        return render_template('add_expense.html', form=form)

    @app.route('/edit/<int:expense_id>', methods=['GET', 'POST'])
    def edit_expense(expense_id):
        if 'user_id' not in session:
            return redirect(url_for('login'))
        expense = query_db('SELECT * FROM expenses WHERE id = ?', [expense_id], one=True)
        if expense['user_id'] != session['user_id']:
            return redirect(url_for('dashboard'))
        form = ExpenseForm(data=expense)
        if form.validate_on_submit():
            db = get_db()
            db.execute('UPDATE expenses SET category = ?, amount = ?, comments = ? WHERE id = ?',
                       (form.category.data, form.amount.data, form.comments.data, expense_id))
            db.commit()
            flash('Expense updated successfully!', 'success')
            return redirect(url_for('dashboard'))
        return render_template('edit_expense.html', form=form)

    @app.route('/delete/<int:expense_id>', methods=['POST'])
    def delete_expense(expense_id):
        if 'user_id' not in session:
            return redirect(url_for('login'))
        expense = query_db('SELECT * FROM expenses WHERE id = ?', [expense_id], one=True)
        if expense['user_id'] == session['user_id']:
            db = get_db()
            db.execute('DELETE FROM expenses WHERE id = ?', [expense_id])
            db.commit()
            flash('Expense deleted successfully!', 'success')
        return redirect(url_for('dashboard'))

    return app

def create_app():
  app = Flask(__name__)
  # ... rest of your create_app function
  # Initialize the database here
  init_db()

  return app

def init_db():
    if __name__ == '__main__':
        app = create_app()
        app.run(debug=True)
